In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json
import google.auth
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

def scrape_page(url):
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    driver.get(url)
    try:
        time.sleep(5)  # Wait for the page to load
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Locate the table within the specified div
        table_div = soup.find('div', class_='col-lg-12 largetable')
        table = table_div.find('table') if table_div else None
        
        extracted_data = []

        if table:
            rows = table.find_all('tr')
            for row in rows:
                row_data = []
                headers = row.find_all('th')
                for header in headers:
                    row_data.append(header.get_text().strip())
                cells = row.find_all('td')
                for cell in cells:
                    row_data.append(cell.get_text().strip())
                extracted_data.append(row_data)
        
        # Write the extracted data to Google Sheet
        write_to_google_sheet(extracted_data)
        
    except Exception as e:
        print("Error:", e)
    finally:
        driver.quit()

def write_to_google_sheet(data):
    # Define the scope and authenticate with the service account
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = 'path_to_your_service_account.json'
    
    credentials = Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    
    # The ID of the spreadsheet
    SPREADSHEET_ID = '1b204kgy52jsyu4ptvFO1Ts9yCbDtwtq6AxoTIjCmbtY'

    # Build the service
    service = build('sheets', 'v4', credentials=credentials)
    
    # Prepare the data to be inserted
    body = {
        'values': data
    }

    # Clear the existing content
    clear_range = 'Sheet1!A:Z'  # Adjust the range as needed
    service.spreadsheets().values().clear(spreadsheetId=SPREADSHEET_ID, range=clear_range).execute()
    
    # Insert the data into the Google Sheet
    result = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID, range='Sheet1!A1',
        valueInputOption='RAW', body=body).execute()
    
    print(f"{result.get('updatedCells')} cells updated.")

# Replace the URL with the target URL
url = 'https://www.bseindia.com/corporates/ann.html'
scrape_page(url)


Error: [Errno 2] No such file or directory: 'path_to_your_service_account.json'
